# Web Scraping con Python - Proyecto de práctica

Este proyecto forma parte del módulo final del curso ¨Python for Data Science, AI & Development" de IBM

---
## Objetivos

- Familiarizarme con el uso de `BeatifulSoup`
- Extracción y filtrado de datos de páginas web
- Comprender la estructura del DOM y cómo navegarla con Python

---
## Contenido 

- Objeto 'BeatifulSoup y su estructura
- Exploración de etiquetas HTML, relaciones entre elementos (padres, hijos, hermanos)
- Uso de atributos HTML y texto navegable
- Métodos `find()` y `find_all()` para buscar información
- Descarga y análisis de contenido web

---
## Requisitos

Se utilizacon bibliotecas de Python como `requests` y `bs4`.
Algunas ya están disponibles en entornos como SN Labs, y otras pueden requerir instalación previa. 

---

Desarrollado por **Germán M. Zepeda**



In [1]:
import os

In [2]:
ruta = r'C:\PROYECTOS\GitHub\Web Scraping con Python'
os.chdir(ruta)
print(f'El directorio de trabajo es: {os.getcwd()}')

El directorio de trabajo es: C:\PROYECTOS\GitHub\Web Scraping con Python


In [3]:
!pip install bs4
!pip install requests pandas html5lib

In [4]:
from bs4 import BeautifulSoup # permite extraer datos de páginas web (web scraping)
import requests # permite descargar una página web

## Objetos de Beautiful Soup
Beautiful Soup es una biblioteca de Python utilizada para extraer datos de archivos HTML y XML. El enfoque va a estar en archivos HTML.

La extracción se logra representando el HTML como un conjunto de objetos, que incluyen métodos para analizar (parsear) el contenido. Esto permite navegar el HTML como si fuera un árbol de elementos, y también filtrar la información que estamos buscando.

Consideremos el siguiente código HTML:
Este es un pequeño fragmento de HTML que se usará para practicar la extracción de datos con BeautifulSoup. Contiene los nombres de tres jugadores de básquet y sus salarios:

In [5]:
%%html
<!DOCTYPE html>
<html>
<head>
<title>Page Title</title>
</head>
<body>
<h3><b id='boldest'>Lebron James</b></h3>
<p> Salary: $ 92,000,000 </p>
<h3> Stephen Curry</h3>
<p> Salary: $85,000, 000 </p>
<h3> Kevin Durant </h3>
<p> Salary: $73,200, 000</p>
</body>
</html>

**Podemos almacenarlo como una cadena de texto en la variable HTML:**

In [6]:
html = "<!DOCTYPE html><html><head><title>Page Title</title></head><body><h3><b id='boldest'>Lebron James</b></h3><p> Salary: $ 92,000,000 </p><h3> Stephen Curry</h3><p> Salary: $85,000, 000 </p><h3> Kevin Durant </h3><p> Salary: $73,200, 000</p></body></html>"

In [10]:
html

"<!DOCTYPE html><html><head><title>Page Title</title></head><body><h3><b id='boldest'>Lebron James</b></h3><p> Salary: $ 92,000,000 </p><h3> Stephen Curry</h3><p> Salary: $85,000, 000 </p><h3> Kevin Durant </h3><p> Salary: $73,200, 000</p></body></html>"

#### Analizar (parsear) un documento con BeautifulSoup
Para analizar un documento HTML, hay que pasarlo como argumento al constructor de BeautifulSoup.
El objeto que se crea representa el documento como una estructura de datos anidada, similar a un árbol.

In [7]:
soup = BeautifulSoup(html, 'html5lib')

**Cómo procesa BeautifulSoup un documento HTML**

Primero, el documento HTML se convierte a Unicode (similar a ASCII, pero más completo), y las entidades HTML (como &nbsp;, &amp;, etc.) se traducen a sus respectivos caracteres.

Luego, BeautifulSoup transforma el HTML en un árbol complejo de objetos de Python, lo que permite navegarlo fácilmente.

- BeautifulSoup y Tag son los dos tipos principales de objetos que vamos a usar (en esta práctica, serán como equivalentes).
- También vamos a ver objetos NavigableString, que representan texto dentro de las etiquetas.

  ---
  Podemos usar el método `.prettify()` para mostrar el HTML con la indentación (sangrado) correcta, como si fuera un árbol visual:

In [12]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Page Title
  </title>
 </head>
 <body>
  <h3>
   <b id="boldest">
    Lebron James
   </b>
  </h3>
  <p>
   Salary: $ 92,000,000
  </p>
  <h3>
   Stephen Curry
  </h3>
  <p>
   Salary: $85,000, 000
  </p>
  <h3>
   Kevin Durant
  </h3>
  <p>
   Salary: $73,200, 000
  </p>
 </body>
</html>



**Etiquetas (Tags)**

Supongamos que queremos obtener el título de la página y el nombre del jugador mejor pago.
Podemos usar un Tag.

El objeto Tag representa una etiqueta HTML del documento original. Por ejemplo, la etiqueta 'tittle'

##### Niveles de encabezado

h1 -> Título principal 

h2 -> Subtítulo

h3 -> Sub-subtítulo

...

h6 -> Encabezado más prequeño y menos importante

In [13]:
# etiqueta_titulo = soup.title       # accede a la etiqueta <title>, ej: <title>Mi Sitio Web</title>
# etiqueta_h1 = soup.h1              # accede a la primera etiqueta <h1>, ej: (si existiera) <h1>Bienvenidos a mi sitio</h1>
# etiqueta_h3 = soup.h3              # accede a la primera etiqueta <h3>
# etiqueta_parrafo = soup.p          # accede al primer <p> (párrafo)
# etiqueta_negrita = soup.b          # accede al primer <b> (negrita)


# Accedemos a la etiqueta <title>
etiqueta_titulo = soup.title
print(f"Título completo (etiqueta <title>): {etiqueta_titulo.text}")

# Accedemos a la primera etiqueta <h1> (en este caso debería ser None, porque no existe en el HTML)
etiqueta_h1 = soup.h1
print(f"Etiqueta <h1>: {etiqueta_h1}")


Título completo (etiqueta <title>): Page Title
Etiqueta <h1>: None


In [15]:
# Podemos ver que el tipo de etiqueta es bs4.element.Tag. Confirma que es una etiqueta HTML

print("etiqueta_titulo type:",type(etiqueta_titulo))

etiqueta_titulo type: <class 'bs4.element.Tag'>


## Hijos (Children), Padres (Parents) y Hermanos (Sibling) en BeautifulSoup

Un objeto `Tag` en BeautifulSoup representa una estructura en forma de **árbol**.  
Esto significa que cada etiqueta puede tener **hijos** (elementos dentro), **padres** (elemento que la contiene), y **hermanos** (elementos al mismo nivel).

Podemos movernos hacia abajo en el árbol (acceder a un hijo), subir (acceder al padre), o desplazarnos lateralmente (ver hermanos).


**Accedemos a la etiqueta `h3`**

In [27]:
etiqueta_h3 = soup.h3
etiqueta_h3.contents # muestra los hijos

[<b id="boldest">Lebron James</b>]

**Accedemos al hijo `<b>` dentro de `<h3`** 

- `.contents` no da su contenido

In [29]:
etiqueta_b = etiqueta_h3.b
etiqueta_b.contents


['Lebron James']

In [33]:
etiqueta_b.text # nos da solo el texto interno

'Lebron James'

**Accedemos al padre usando `.parent`**

In [31]:
etiqueta_padre = etiqueta_h3.parent
etiqueta_padre

<body><h3><b id="boldest">Lebron James</b></h3><p> Salary: $ 92,000,000 </p><h3> Stephen Curry</h3><p> Salary: $85,000, 000 </p><h3> Kevin Durant </h3><p> Salary: $73,200, 000</p></body>

#### Usamos `.parent` para acceder al elemento padre de `etiqueta_h3`, que es `<body>`.

Luego usamos `.next_sibling` para ir a los hermanos:
- `hermana_1` → es el `<p>` con el salario de Lebron
- `hermana_2` → es el siguiente `<h3>`, que contiene a Stephen Curry


In [34]:
# Su padre es la etiqueta <body>
print(f"Padre de etiqueta_h3: {etiqueta_h3.parent.name}")  # body


Padre de etiqueta_h3: body


In [35]:
# Su primer hermano (next_sibling) es el párrafo con el salario
etiqueta_hermana_1 = etiqueta_h3.next_sibling
print(f"Primer hermano (hermana_1): {etiqueta_hermana_1}")


Primer hermano (hermana_1): <p> Salary: $ 92,000,000 </p>


In [36]:
# Su segundo hermano (otro <h3>)
etiqueta_hermana_2 = etiqueta_hermana_1.next_sibling
print(f"Segundo hermano (hermana_2): {etiqueta_hermana_2}")


Segundo hermano (hermana_2): <h3> Stephen Curry</h3>


## Ejercicio: uso de `.next_sibling` para acceder al hermano siguiente

Usá el objeto etiqueta_hermana_2 y el método `.next_sibling` para encontrar el salario de Stephen Curry:

In [39]:
salario_steohen_curry = etiqueta_hermana_2.next_sibling
print(f'El salario de Stephen Curry es de: {salario_steohen_curry}')

El salario de Stephen Curry es de: <p> Salary: $85,000, 000 </p>


### Atributos HTML
Si una etiqueta tiene atributos, como en este caso id="boldest", el atributo es id y su valor es "boldest".
Se puede acceder a los atributos de una etiqueta **tratándola como un diccionario** en Python.


In [44]:
etiqueta_hijo = etiqueta_h3.b

In [45]:
etiqueta_hijo

<b id="boldest">Lebron James</b>

In [46]:
etiqueta_hijo['id']

'boldest'

**También se puede acceder a ese diccionario directamente usando `.attrs`**

In [47]:
etiqueta_hijo.attrs

{'id': 'boldest'}

**Además, se puede obtener el contenido de un atributo de una etiqueta utilizando el método `get()` de Python**

In [48]:
etiqueta_hijo.get('id')

'boldest'

### NavigableString

Una cadena de texto (string) corresponde a un fragmento de texto o contenido dentro de una etiqueta.
BeautifulSoup utiliza la clase NavigableString para representar ese texto.
En nuestro HTML, podemos obtener el nombre del primer jugador extrayendo la cadena de texto del objeto Tag <code>etiqueta_hijo</code>, de la siguiente manera:


In [49]:
texto_jugador = etiqueta_hijo.string
print(texto_jugador)


Lebron James


In [50]:
type(texto_jugador)

bs4.element.NavigableString

Un **NavigableString** es similar a una cadena de texto en Python (tipo str o unicode).
La diferencia es que también incluye algunas funcionalidades específicas de BeautifulSoup.
Podemos convertir un NavigableString en una cadena normal de Python usando `str()`

In [56]:
jugador_string = str(texto_jugador)
print(jugador_string)


Lebron James


## Filtros
Los filtros permiten encontrar patrones complejos.
El filtro más simple es una cadena de texto.

En esta sección se utilizará una cadena como filtro en distintos métodos, y **BeautifulSoup** buscará coincidencias exactas con ese texto.

Se utilizará el siguiente HTML de lanzamientos de cohetes como ejemplo:

In [8]:
%%html
<table>
  <tr>
    <td id='flight' >Flight No</td>
    <td>Launch site</td> 
    <td>Payload mass</td>
   </tr>
  <tr> 
    <td>1</td>
    <td><a href='https://en.wikipedia.org/wiki/Florida'>Florida</a></td>
    <td>300 kg</td>
  </tr>
  <tr>
    <td>2</td>
    <td><a href='https://en.wikipedia.org/wiki/Texas'>Texas</a></td>
    <td>94 kg</td>
  </tr>
  <tr>
    <td>3</td>
    <td><a href='https://en.wikipedia.org/wiki/Florida'>Florida<a> </td>
    <td>80 kg</td>
  </tr>
</table>

Flight No,Launch site,Payload mass
1,Florida,300 kg
2,Texas,94 kg
3,Florida,80 kg


In [9]:
tabla="<table><tr><td id='flight'>Flight No</td><td>Launch site</td> <td>Payload mass</td></tr><tr> <td>1</td><td><a href='https://en.wikipedia.org/wiki/Florida'>Florida<a></td><td>300 kg</td></tr><tr><td>2</td><td><a href='https://en.wikipedia.org/wiki/Texas'>Texas</a></td><td>94 kg</td></tr><tr><td>3</td><td><a href='https://en.wikipedia.org/wiki/Florida'>Florida<a> </td><td>80 kg</td></tr></table>"

In [10]:
tabla_bs=BeautifulSoup(tabla, 'html5lib')

### find_all()
El método `find_all()` recorre los descendientes de una etiqueta y recupera todos los elementos que coincidan con los filtros especificados.
find_all(name, attrs, recursive, string, limit, **kwargs)
### Parámetro **name**
Cuando se usa el parámetro **name** con el nombre de una etiqueta (por ejemplo "td"), el método devuelve todas las etiquetas con ese nombre, junto con sus elementos hijos.


In [11]:
tabla_fila=tabla_bs.find_all('tr')
tabla_fila

[<tr><td id="flight">Flight No</td><td>Launch site</td> <td>Payload mass</td></tr>,
 <tr> <td>1</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a></a></td><td>300 kg</td></tr>,
 <tr><td>2</td><td><a href="https://en.wikipedia.org/wiki/Texas">Texas</a></td><td>94 kg</td></tr>,
 <tr><td>3</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a> </a></td><td>80 kg</td></tr>]

**El resultado es un Iterable de Python, similar a una lista.
Cada elemento es un objeto `Tag`.**

In [12]:
primera_fila=tabla_fila[0]
primera_fila

<tr><td id="flight">Flight No</td><td>Launch site</td> <td>Payload mass</td></tr>

In [13]:
print(type(primera_fila))

<class 'bs4.element.Tag'>


**Podemos obtener el hijo (child).**

In [14]:
primera_fila.td

<td id="flight">Flight No</td>

**Si iteramos sobre la lista, cada elemento corresponde a una fila de la tabla.**

In [15]:
for i,fila in enumerate(tabla_fila):
    print("Fila",i,"en", fila)

Fila 0 en <tr><td id="flight">Flight No</td><td>Launch site</td> <td>Payload mass</td></tr>
Fila 1 en <tr> <td>1</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a></a></td><td>300 kg</td></tr>
Fila 2 en <tr><td>2</td><td><a href="https://en.wikipedia.org/wiki/Texas">Texas</a></td><td>94 kg</td></tr>
Fila 3 en <tr><td>3</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a> </a></td><td>80 kg</td></tr>


**Como *fila* es un objeto Tag, se le puede aplicar el `método find_all` para extraer las celdas de la tabla (etiquetas `td`).
Esto devuelve todos los hijos con el nombre td.**

**El resultado es una lista, donde cada elemento es una celda representada por un objeto `Tag`.
También se puede iterar sobre esa lista y extraer el contenido usando el atributo .string.**

In [26]:
for i, fila in enumerate(tabla_fila):
    print('Fila', i)
    celdas = fila.find_all('td')
    for j, celda in enumerate(celdas):
        print('Columna', j, 'Celda:', celda.string)



Fila 0
Columna 0 Celda: Flight No
Columna 1 Celda: Launch site
Columna 2 Celda: Payload mass
Fila 1
Columna 0 Celda: 1
Columna 1 Celda: None
Columna 2 Celda: 300 kg
Fila 2
Columna 0 Celda: 2
Columna 1 Celda: Texas
Columna 2 Celda: 94 kg
Fila 3
Columna 0 Celda: 3
Columna 1 Celda: None
Columna 2 Celda: 80 kg


**Si usamos una lista, podemos hacer coincidir contra cualquier elemento de esa lista.**

In [27]:
lista_salida=tabla_bs.find_all(name=['tr','td'])
lista_salida

[<tr><td id="flight">Flight No</td><td>Launch site</td> <td>Payload mass</td></tr>,
 <td id="flight">Flight No</td>,
 <td>Launch site</td>,
 <td>Payload mass</td>,
 <tr> <td>1</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a></a></td><td>300 kg</td></tr>,
 <td>1</td>,
 <td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a></a></td>,
 <td>300 kg</td>,
 <tr><td>2</td><td><a href="https://en.wikipedia.org/wiki/Texas">Texas</a></td><td>94 kg</td></tr>,
 <td>2</td>,
 <td><a href="https://en.wikipedia.org/wiki/Texas">Texas</a></td>,
 <td>94 kg</td>,
 <tr><td>3</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a> </a></td><td>80 kg</td></tr>,
 <td>3</td>,
 <td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a> </a></td>,
 <td>80 kg</td>]

## Atributos
Si el argumento que se pasa a `find_all()` no es reconocido como un parámetro estándar, se interpreta como un filtro sobre los atributos de las etiquetas.

Por ejemplo, si se pasa el argumento `id`, BeautifulSoup filtrará por el atributo `id` de cada etiqueta.

En este caso, el primer elemento <td> tiene el atributo `id="flight"`, por lo tanto se puede aplicar un filtro basado en ese valor.

In [16]:
tabla_bs.find_all(id='flight')

[<td id="flight">Flight No</td>]

**Podemos encontrar todos los elementos que contienen enlaces a la página de Wikipedia de Florida:**

In [29]:
lista_salida=tabla_bs.find_all(href='https://en.wikipedia.org/wiki/Florida')
lista_salida

[<a href="https://en.wikipedia.org/wiki/Florida">Florida</a>,
 <a href="https://en.wikipedia.org/wiki/Florida">Florida</a>]

**Si configuramos el atributo href en True, sin importar cuál sea su valor, el código encuentra todas las etiquetas que tengan un atributo href.**

In [30]:
# El atributo "href" nos permite entontrar todos los enlaces en un documento HTML.

tabla_bs.find_all(href=True)

[<a href="https://en.wikipedia.org/wiki/Florida">Florida</a>,
 <a href="https://en.wikipedia.org/wiki/Texas">Texas</a>,
 <a href="https://en.wikipedia.org/wiki/Florida">Florida</a>]

## Ejercicio: `find_all()`

- Usando la lógica anterior encontrar todos los elementos que **no tengan** atributo `href`.
- Usando el objeto `soup`, encontrar el elemento cuyo atributo `id` tenga el valor `"boldest"`.
- Con el parámetro `string` buscar textos en lugar de etiquetas. Por ej.: podemos encontrar todos los elementos que contengan el texto **Florida**.

In [32]:
tabla_bs.find_all(href=False)

[<html><head></head><body><table><tbody><tr><td id="flight">Flight No</td><td>Launch site</td> <td>Payload mass</td></tr><tr> <td>1</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a></a></td><td>300 kg</td></tr><tr><td>2</td><td><a href="https://en.wikipedia.org/wiki/Texas">Texas</a></td><td>94 kg</td></tr><tr><td>3</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a> </a></td><td>80 kg</td></tr></tbody></table></body></html>,
 <head></head>,
 <body><table><tbody><tr><td id="flight">Flight No</td><td>Launch site</td> <td>Payload mass</td></tr><tr> <td>1</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a></a></td><td>300 kg</td></tr><tr><td>2</td><td><a href="https://en.wikipedia.org/wiki/Texas">Texas</a></td><td>94 kg</td></tr><tr><td>3</td><td><a href="https://en.wikipedia.org/wiki/Florida">Florida</a><a> </a></td><td>80 kg</td></tr></tbody></table></body>,
 <table><tbody><tr><td id="flight">Flight No</td><td>Launch site</td> <t

**En el primer html**:

In [40]:
soup.find_all(id="boldest")

[<b id="boldest">Lebron James</b>]

In [41]:
tabla_bs.find_all(string="Florida")

['Florida', 'Florida']

## find() vs. find_all()

El método `find_all()` recorre **todo el documento** buscando todos los resultados que coincidan. 
Pero si solo se necesita el **primer elemento**, es mejor usar el método `find()`, que devuelve la **primera coincidencia** encontrada.

A continuación se presentan **dos tablas** como ejemplo:

In [17]:
%%html
<h3>Rocket Launch </h3>

<p>
<table class='rocket'>
  <tr>
    <td>Flight No</td>
    <td>Launch site</td> 
    <td>Payload mass</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Florida</td>
    <td>300 kg</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Texas</td>
    <td>94 kg</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Florida </td>
    <td>80 kg</td>
  </tr>
</table>
</p>
<p>

<h3>Pizza Party  </h3>
  
    
<table class='pizza'>
  <tr>
    <td>Pizza Place</td>
    <td>Orders</td> 
    <td>Slices </td>
   </tr>
  <tr>
    <td>Domino's Pizza</td>
    <td>10</td>
    <td>100</td>
  </tr>
  <tr>
    <td>Little Caesars</td>
    <td>12</td>
    <td >144 </td>
  </tr>
  <tr>
    <td>Papa John's </td>
    <td>15 </td>
    <td>165</td>
  </tr>

Flight No,Launch site,Payload mass
1,Florida,300 kg
2,Texas,94 kg
3,Florida,80 kg
Pizza Place,Orders,Slices
Domino's Pizza,10,100
Little Caesars,12,144
Papa John's,15,165


In [18]:
two_tables="<h3>Rocket Launch </h3><p><table class='rocket'><tr><td>Flight No</td><td>Launch site</td> <td>Payload mass</td></tr><tr><td>1</td><td>Florida</td><td>300 kg</td></tr><tr><td>2</td><td>Texas</td><td>94 kg</td></tr><tr><td>3</td><td>Florida </td><td>80 kg</td></tr></table></p><p><h3>Pizza Party  </h3><table class='pizza'><tr><td>Pizza Place</td><td>Orders</td> <td>Slices </td></tr><tr><td>Domino's Pizza</td><td>10</td><td>100</td></tr><tr><td>Little Caesars</td><td>12</td><td >144 </td></tr><tr><td>Papa John's </td><td>15 </td><td>165</td></tr>"

In [19]:
# Creamos el objeto BeautifulSoup

two_tables_bs=BeautifulSoup(two_tables, 'html.parser') # 'html.parser' viene instalado con Python. Es más rápido y
                                                       # menos tolerante a errores. 

In [20]:
# Con el método 'find()' podemos acceder a la primera tabla utilizando la etiqueta 'table'.

first_table=two_tables_bs.find("table")
print(first_table)

<table class="rocket"><tr><td>Flight No</td><td>Launch site</td> <td>Payload mass</td></tr><tr><td>1</td><td>Florida</td><td>300 kg</td></tr><tr><td>2</td><td>Texas</td><td>94 kg</td></tr><tr><td>3</td><td>Florida </td><td>80 kg</td></tr></table>


In [21]:
# Podemos filtrar por el atributo 'class' para encontrar la segunda tabla.
# Pero como 'class' es una palabra reservada de Python, le agregamos un guin bajo: class_ 

second_table=two_tables_bs.find('table', class_='pizza')
print(second_table)

<table class="pizza"><tr><td>Pizza Place</td><td>Orders</td> <td>Slices </td></tr><tr><td>Domino's Pizza</td><td>10</td><td>100</td></tr><tr><td>Little Caesars</td><td>12</td><td>144 </td></tr><tr><td>Papa John's </td><td>15 </td><td>165</td></tr></table>


## Descarga y extracción del contenido de una página web

In [27]:
# Descargamos el coontenido de la página web:

url="https://www.ibm.com"

In [28]:
# Usamos requests.get() para descargar el contenido de la página web en formato de texto y lo almacenamos en una 
# variable llamada 'data'.

data=requests.get(url).text

In [29]:
# Creamos un objeto BeautifulSoup utilizando el constructor de BeautifulSoup
soup = BeautifulSoup(data, 'html5lib')

In [30]:
# Extraemos todos los enlaces

for link in soup.find_all('a',href=True): # # en HTML, un enlace se representa con la etiqueta <a>

    print(link.get('href'))

https://www.ibm.com/finops?lnk=hpls1us
https://www.ibm.com/products/turbonomic/cloud-optimization?lnk=hpls2us
https://www.ibm.com/granite?lnk=hpdev1us
https://developer.ibm.com/technologies/artificial-intelligence?lnk=hpdev2us
https://skillsbuild.org/?lnk=hpdev3us
https://www.youtube.com/watch?v=g0CaNTs5DaI&lnk=hpdev4us
https://www.ibm.com/new/announcements/ibm-named-a-leader-in-the-2025-gartner-magic-quadrant-for-data-science-and-machine-learning-platforms?lnk=hpdev5us
https://www.ibm.com/new/announcements/ibm-leader-2025-omdia-universe-on-no-low-pro-ide-assistants-report?lnk=hpdev6us
https://www.ibm.com/products/watsonx-code-assistant/pricing?lnk=hpdev7us
https://www.ibm.com/products/watsonx-ai?lnk=hpdev8us
https://www.ibm.com/products/offers-and-discounts?lnk=hpdo1us
https://www.ibm.com/artificial-intelligence?lnk=hpfp1us
https://www.ibm.com/hybrid-cloud?lnk=hpfp2us
https://www.ibm.com/consulting?lnk=hpfp3us
https://www.ibm.com/about?lnk=hpii1us
https://www.ibm.com/history?lnk=hpii1

### En HTML, las etiquetas (tags) como `img` se usan para representar imágenes.
Vamos a extraer todas las tags `img` del documento.

In [33]:
for link in soup.find_all('img'): # en HTML imagen se representa con la etiqueta <img>
    print(link)
    print(link.get('src'))

In [34]:
# Para verificar si hay imágenes

imagenes = soup.find_all("img")
print(f"Cantidad de imágenes encontradas: {len(imagenes)}")


Cantidad de imágenes encontradas: 0


## Extracción de datos desde tablas en HTML

Antes de proceder a extraer datos de un sitio web, es necesario examinar su contenido y cómo está organizada la información.
Verificar previamente cuántas filas y columnas tiene la tabla de colores.

In [35]:
# La URL de abajo contiene una tabla HTML con información sobre colores y sus códigos.
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DA0321EN-SkillsNetwork/labs/datasets/HTMLColorCodes.html"

In [42]:
# Obtener el contenido de la página web en formato de texto y guardarlo en una variable llamada data
data=requests.get(url).text

In [43]:
soup=BeautifulSoup(data, 'html5lib')

In [49]:
table=soup.find('table') # en HTML una tabla se representa con la etiqueta <table>

In [50]:
# Obtener todas las filas de la tabla
for row in table.find_all('tr'): # en HTML, una fila de tabla se representa con la etiqueta <tr>
    # Obtener todas las columnas en cada fila
    cols = row.find_all('td') # en HTML a la columna se la representa con la etiqueta <td>
    color_name = cols[2].string # guardar el valor de la columna 3 como color_name
    color_code = cols[3].string # guardar el valor de la columna 4 como color_code
    print("{}--->{}".format(color_name, color_code))


Color Name--->None
lightsalmon--->#FFA07A
salmon--->#FA8072
darksalmon--->#E9967A
lightcoral--->#F08080
coral--->#FF7F50
tomato--->#FF6347
orangered--->#FF4500
gold--->#FFD700
orange--->#FFA500
darkorange--->#FF8C00
lightyellow--->#FFFFE0
lemonchiffon--->#FFFACD
papayawhip--->#FFEFD5
moccasin--->#FFE4B5
peachpuff--->#FFDAB9
palegoldenrod--->#EEE8AA
khaki--->#F0E68C
darkkhaki--->#BDB76B
yellow--->#FFFF00
lawngreen--->#7CFC00
chartreuse--->#7FFF00
limegreen--->#32CD32
lime--->#00FF00
forestgreen--->#228B22
green--->#008000
powderblue--->#B0E0E6
lightblue--->#ADD8E6
lightskyblue--->#87CEFA
skyblue--->#87CEEB
deepskyblue--->#00BFFF
lightsteelblue--->#B0C4DE
dodgerblue--->#1E90FF
